# 🎬 YouTube Comment Moderator AI

## Tone Classification & Summary Workflow

---

### 🚦 Moderation Flow

1️⃣ **User enters YouTube link**

2️⃣ **System fetches video comments**

3️⃣ **AI classifies comments**
   - Toxicity
   - Spam
   - Hate
   - Positive
   - Neutral

4️⃣ **AI summarizes overall sentiment & issues**
   - Highlights dominant tones
   - Flags problematic trends
   - Provides actionable insights

5️⃣ **Show moderation report**
   - Visual summary of tone distribution
   - Key findings & recommendations

---

> **Example Output:**
>
> - **Tone Distribution:** ![Pie Chart Icon](https://img.icons8.com/color/48/000000/pie-chart.png)
> - **Summary:** "Most comments are positive, but 12% show signs of toxicity. Spam detected in 8%."
> - **Recommendations:** "Consider moderating toxic and spam comments. Overall sentiment is positive."

---

✨ Use this workflow to generate other cells for each step, making your notebook interactive and visually appealing!

---

## STEP 1: Import Required Libraries

Below are the libraries needed for the YouTube Comment Moderator AI, as listed in `requirements.txt`:

- **google-api-python-client**: For accessing YouTube API and fetching comments
- **pandas**: For data manipulation and analysis
- **nltk**: For natural language processing and tone classification
- **scikit-learn**: For machine learning models and sentiment analysis
- **ollama**: For advanced AI and LLM integration

> **Next:** Add a code cell to import these libraries and check their installation.

In [4]:


# Import required libraries for YouTube Comment Moderator AI
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import ollama  


In [5]:
video_url = "https://www.youtube.com/watch?v=3lS6ITUXQAI"  # Example video URL

##  STEP 2: Extract Video ID from URL

In [6]:
import re
def get_video_id(video_url):
    pattern = r"(?:v=|\/)([0-9A-Za-z_-]{11}).*"
    match = re.search(pattern, video_url)
    return match.group(1) if match else None

video_id = get_video_id(video_url)
video_id

'3lS6ITUXQAI'

##  STEP 2: Fetch Comments from YouTube with your youtube API keys

In [7]:
load_dotenv(override=True)

youtube_api_key = os.getenv("YOUTUBE_API_KEY")

if not youtube_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not youtube_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")


No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini


In [ ]:


youtube = build('youtube', 'v3', developerKey=youtube_api_key)

def get_comments(video_id, max_comments=100):
    comments = []
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=100,
        textFormat="plainText"
    )

    response = request.execute()

    while request and len(comments) < max_comments:
        for item in response["items"]:
            comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
            comments.append(comment)

        if "nextPageToken" in response:
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                pageToken=response["nextPageToken"],
                maxResults=100,
                textFormat="plainText"
            )
            response = request.execute()
        else:
            break

    return comments

comments = get_comments(video_id, 500)
comments[:5]

##  STEP 3: Classify Comments using Ollama

In [ ]:


def classify_comment(comment):
    user_prompt = f"""
    Classify this YouTube comment into one category:

    Categories:
    - Positive
    - Neutral
    - Toxic
    - Spam
    - Hate Speech

    Comment: {comment}

    Return only the category name.
    """

    response = ollama.chat(
        model='gemma3:1b',
        messages=[{"role": "user", "content": user_prompt}]
    )

    return response['message']['content'].strip()

# test
classify_comment("This video is amazing!")

## Step 4: Classify All Comments

In [ ]:
results = []

for comment in comments:
    label = classify_comment(comment)
    results.append((comment, label))

results[:5]

## Step 5: Convert to DataFrame

In [ ]:
import pandas as pd

df = pd.DataFrame(results, columns=["Comment", "Category"])
df.head()

## Step 6: Moderation Summary

In [ ]:
summary_counts = df["Category"].value_counts()
summary_counts

## Step 7: AI Summary of Moderation

In [ ]:
def summarize_results(summary):
    system_prompt = f"""
    Summarize the moderation results:

    {summary.to_dict()}

    Explain the overall sentiment and moderation concerns.
    """

    response = ollama.chat(
        model='gemma3:1b',
        messages=[{"role": "user", "content": system_prompt}]
    )

    return response['message']['content']

summarize_results(summary_counts)